# Workshop 2: The Invoice Analyzer Agent

Imagine this: Your finance team dumps **7 invoices** on your desk. *"We need an expense report by end of day."* Each invoice looks different — different vendors, different formats. Some are text files, some are PDFs. You need to read each one, pull out the key details, and put them in a nice spreadsheet.

Sounds tedious? **Let's make an AI do it.**

In this workshop, you'll watch an AI agent read messy invoices and produce a clean CSV expense report — all by itself. All code is pre-written. You just run cells and observe.

Let's start by setting things up.

In [ ]:
# Just run this cell! Press Shift+Enter
# This connects us to the AI service

import os
import csv
import json
from openai import OpenAI

API_KEY = os.getenv("API_KEY") or os.getenv("OPENAI_API_KEY") or os.getenv("ANTHROPIC_API_KEY") or os.getenv("GEMINI_API_KEY")

if API_KEY and API_KEY.startswith("sk-ant-"):
    BASE_URL = "https://api.anthropic.com/v1/"
    MODEL = "claude-sonnet-4-20250514"
elif API_KEY and API_KEY.startswith("sk-"):
    BASE_URL = "https://api.openai.com/v1"
    MODEL = "gpt-4o"
else:
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    MODEL = "gemini-2.0-flash"

BASE_URL = os.getenv("OPENAI_BASE_URL", BASE_URL)
MODEL = os.getenv("MODEL", MODEL)
MAX_TOKENS = 4096
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

print(f"Connected! Using model: {MODEL}")

---

## Section 1: The Problem — A Stack of Invoices

Let's start by looking at what we're dealing with. There's a folder called `invoices/` sitting right next to this notebook. It's full of invoice files from different vendors.

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's see what invoice files we have

import os

invoice_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "invoices")
files = sorted(os.listdir(invoice_dir))

print(f"Found {len(files)} invoice files:\n")
for f in files:
    size = os.path.getsize(os.path.join(invoice_dir, f))
    print(f"  {f}  ({size} bytes)")

7 invoices. Some are `.txt` files, some are `.pdf` files. Let's peek inside one to see what unstructured data looks like.

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's look at one invoice to see what we're working with

with open(os.path.join(invoice_dir, "invoice_1001.txt")) as f:
    contents = f.read()

print(contents)

This is **unstructured data**. The numbers, dates, and vendor names are all there — but buried in free-form text. A spreadsheet can't read this. There are no neat columns. No consistent delimiters.

But an AI can read it the same way you would — it just *understands* the text.

Now imagine doing this for hundreds of invoices, each with a slightly different layout. That's the problem we're solving.

---

## Section 2: The Game Plan — How Would YOU Do It?

Before we build anything, let's think like a PM. If **you** were processing these invoices manually, what would your steps be?

Take a moment to think about it before reading on.

...

Here's what most people come up with:

1. **Look** at what invoice files exist in the folder
2. **Open** each invoice and read it
3. **Pull out** the key details: invoice number, date, vendor name, total amount
4. **Write** each expense as a row in a spreadsheet
5. **Save** the spreadsheet

Now here's the key insight:

> **We give these EXACT steps to the AI as tools.** Each step becomes a tool the AI can use.

The AI doesn't need to know *how* to list files or write CSVs. It just needs to know that those capabilities exist and when to use them. Our code handles the "how." The AI handles the "what" and "when."

---

## Section 3: Designing the Toolbox

Here's where your **PM thinking** matters. We need to decide: what tools does the AI need? And for each tool: what information does it need to do its job?

We're going to give the AI **4 tools**:

| Tool | What it does | What it needs |
|------|-------------|---------------|
| **list_invoices** | Look at what's in the folder | Nothing — just look |
| **read_invoice** | Open and read one invoice | Which file to read |
| **add_expense** | Record one expense line | Invoice number, date, vendor, amount |
| **save_report** | Save everything to a CSV file | What to name the file |

Think of this like designing an API for a junior employee. You're saying: *"Here are the actions you can take, and here's what information you need to provide for each one."*

Let's define these tools in a format the AI understands.

In [ ]:
# Just run this cell! Press Shift+Enter
# These are the tool definitions — the "menu" of capabilities we give the AI

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "list_invoices",
            "description": "List all invoice files in the invoices/ directory. Returns a list of filenames.",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_invoice",
            "description": "Read the contents of a specific invoice file. Returns the full text of the invoice.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {
                        "type": "string",
                        "description": "The invoice filename (e.g. 'invoice_1001.txt')",
                    }
                },
                "required": ["filename"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_expense",
            "description": "Record a single expense extracted from an invoice. Call this once per invoice after reading it.",
            "parameters": {
                "type": "object",
                "properties": {
                    "invoice_number": {
                        "type": "string",
                        "description": "The invoice number (e.g. '1001')",
                    },
                    "date": {
                        "type": "string",
                        "description": "The invoice date in YYYY-MM-DD format",
                    },
                    "vendor": {
                        "type": "string",
                        "description": "The vendor/company name from the 'From:' line",
                    },
                    "amount": {
                        "type": "number",
                        "description": "The total amount in rupees (numeric, no currency symbol or commas)",
                    },
                },
                "required": ["invoice_number", "date", "vendor", "amount"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "save_report",
            "description": "Save all recorded expenses to a CSV file. Call this after all invoices have been processed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {
                        "type": "string",
                        "description": "The output CSV filename (e.g. 'expense_report.csv')",
                    }
                },
                "required": ["filename"],
            },
        },
    },
]

print("Toolbox defined! The AI now has 4 tools available.")
for tool in TOOLS:
    name = tool["function"]["name"]
    desc = tool["function"]["description"]
    params = list(tool["function"]["parameters"]["properties"].keys())
    param_str = "(" + ", ".join(params) + ")" if params else "()"
    print(f"  {name}{param_str} — {desc}")

Read the tool descriptions carefully. These are the **instructions the AI reads** to understand what each tool does. As a PM, writing good tool descriptions is like writing good product requirements — if the description is vague, the AI will be confused.

> **PM Insight:** Notice the `add_expense` tool has `amount` as a **number** type, not **string**. This matters! If we said "string", the AI would send `"8,673"` as text. With "number", it sends `8673` as a clean number we can add up in a spreadsheet. **Type decisions are product decisions.**

---

## Section 4: The Tool Code — What Happens Behind the Scenes

Your engineers write the code that actually **does the work** when a tool is called. The AI doesn't run these functions directly — it says *"I want to use add_expense with these details"* and our code runs the function.

**AI decides WHAT. Code does HOW.**

Let's define what each tool actually does.

In [ ]:
# Just run this cell! Press Shift+Enter
# These are the tool implementations — the actual code that runs

# This list accumulates expense records as the agent processes invoices
expense_records = []

INVOICE_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")), "invoices")


def list_invoices() -> str:
    """List all invoice files in the invoices/ directory."""
    files = sorted(f for f in os.listdir(INVOICE_DIR) if f.endswith(".txt"))
    if not files:
        return "No invoice files found."
    return "\n".join(files)


def read_invoice(filename: str) -> str:
    """Read and return the contents of an invoice file."""
    filepath = os.path.join(INVOICE_DIR, filename)
    with open(filepath) as f:
        return f.read()


def add_expense(invoice_number: str, date: str, vendor: str, amount: float) -> str:
    """Record an expense by appending it to the expense_records list."""
    expense_records.append({
        "invoice_number": invoice_number,
        "date": date,
        "vendor": vendor,
        "amount": amount,
    })
    return f"Recorded expense: Invoice #{invoice_number} from {vendor} for \u20b9{amount:,.0f}"


def save_report(filename: str) -> str:
    """Write all expense records to a CSV file."""
    if not expense_records:
        return "Error: No expenses recorded yet. Process invoices first."
    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["invoice_number", "date", "vendor", "amount"])
        writer.writeheader()
        writer.writerows(expense_records)
    return f"Saved {len(expense_records)} expenses to {filename}"


def execute_tool(name: str, args: dict) -> str:
    """Dispatch a tool call to the right function."""
    if name == "list_invoices":
        return list_invoices()
    elif name == "read_invoice":
        return read_invoice(args["filename"])
    elif name == "add_expense":
        return add_expense(args["invoice_number"], args["date"], args["vendor"], args["amount"])
    elif name == "save_report":
        return save_report(args["filename"])
    else:
        return f"Unknown tool: {name}"


print("Tool implementations loaded!")
print("  list_invoices()   — lists .txt files in invoices/")
print("  read_invoice()    — reads file contents")
print("  add_expense()     — appends to expense_records list")
print("  save_report()     — writes CSV using csv.DictWriter")
print("  execute_tool()    — dispatcher that routes tool calls")

Notice the pattern: each tool function does one simple thing and returns a text result. The AI reads that result and decides what to do next.

Also notice `expense_records = []` — this is our shared data store. As the agent processes each invoice, it calls `add_expense` which adds a row to this list. At the end, `save_report` dumps the whole list into a CSV. The AI never touches this list directly — it only interacts through the tools.

---

## Section 5: The System Prompt — Briefing Your AI Assistant

Before the AI starts working, we give it a briefing. This is like **onboarding a new hire** — you tell them who they are, what the job is, and how to do it step by step.

In [ ]:
# Just run this cell! Press Shift+Enter
# This is the system prompt — the AI's "job description"

SYSTEM_PROMPT = """You are an expense analyzer agent. Your job is to:

1. List all invoice files in the invoices/ directory
2. Read each invoice file one by one
3. Extract the key details: invoice number, date, vendor name, and total amount
4. Record each expense using the add_expense tool
5. After processing ALL invoices, save the report to a CSV file called "expense_report.csv"

Important rules:
- Process every invoice file you find (files ending in .txt)
- Extract the TOTAL amount (not subtotal) from each invoice
- Use the exact vendor name from the "From:" line
- When done with all invoices, save the report as "expense_report.csv"
"""

print("System prompt set!")
print()
print(SYSTEM_PROMPT)

Read it again. We told the AI **exactly what to do, in what order**. Good system prompts are SPECIFIC. Bad ones are vague.

> **Think about it:** What would happen if we just said *"Process the invoices"*? Would the AI know to save a CSV? Would it know to use the TOTAL and not the SUBTOTAL? Probably not. Details matter. Vague instructions produce vague results.

---

## Section 6: Watch the Agent Work!

This is the big moment. We're going to let the AI loose on those invoices.

Watch carefully — you'll see the AI **thinking, choosing tools, and processing each invoice step by step**. It's going to:
1. List the invoice files
2. Read each one
3. Extract the data
4. Record each expense
5. Save the final report

All autonomously. No human in the loop.

In [ ]:
# Just run this cell! Press Shift+Enter
# Watch the agent process all invoices autonomously!

# Reset the expense records (in case you run this cell again)
expense_records.clear()

# The task message kicks off the agent
task = "Please analyze all invoices in the invoices/ directory and create an expense report CSV."

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": task},
]

print("Expense Analyzer Agent")
print("=" * 60)
print(f"Task: {task}")
print("=" * 60)
print()

step = 0

while True:
    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        tools=TOOLS,
        messages=messages,
    )

    message = response.choices[0].message
    messages.append(message)

    if message.tool_calls:
        for tool_call in message.tool_calls:
            step += 1
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            # Format the tool call nicely
            if args:
                args_str = ", ".join(f'{k}={repr(v)}' for k, v in args.items())
                print(f"Step {step}: AI calls {name}({args_str})")
            else:
                print(f"Step {step}: AI calls {name}()")

            result = execute_tool(name, args)

            # Show result (truncate long outputs)
            result_preview = result if len(result) < 200 else result[:200] + "..."
            print(f"  -> {result_preview}")
            print()

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            })
    else:
        # No more tool calls — the agent is done
        print("=" * 60)
        print("Agent finished!")
        print("=" * 60)
        if message.content:
            print()
            print("Agent's summary:")
            print(message.content)
        break

print(f"\nTotal steps: {step}")
print(f"Expenses recorded: {len(expense_records)}")

The AI just read 5 invoices, extracted data from unstructured text, and produced a structured report.

You didn't write any parsing rules. No *"if line starts with Total:"* code. No regular expressions. No format-specific logic. **The AI just... understood.**

That's the difference between traditional automation and AI agents.

---

## Section 7: The Result — Open the CSV

Let's see what the agent produced.

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's see the expense report the agent created

print("expense_report.csv")
print("=" * 60)

try:
    with open("expense_report.csv") as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    # Print as a nice table
    print(f"{'Invoice':<10} {'Date':<14} {'Vendor':<30} {'Amount':>10}")
    print("-" * 66)

    total = 0
    for row in rows:
        amt = float(row['amount'])
        total += amt
        print(f"{row['invoice_number']:<10} {row['date']:<14} {row['vendor']:<30} \u20b9{amt:>9,.0f}")

    print("-" * 66)
    print(f"{'TOTAL':<10} {'':<14} {'':<30} \u20b9{total:>9,.0f}")
    print()
    print(f"{len(rows)} expenses totaling \u20b9{total:,.0f}")

except FileNotFoundError:
    print("expense_report.csv not found. Run the agent cell above first!")

From messy text files to a clean, structured spreadsheet. **That's the power of AI agents.**

Each row was extracted by the AI reading natural language text and understanding what the invoice number, date, vendor, and total amount were — without any hardcoded parsing rules.

---

## Section 8: The Aha Moment — Why This Matters

Let's step back and think about what just happened.

**Traditional automation (the old way):**
- You write parsing rules for EVERY invoice format
- New vendor with a different layout? Write new rules
- Different currency format? More rules
- PDF instead of text? Completely different code
- It's fragile — one unexpected format breaks everything

**AI agent (what we just built):**
- You describe WHAT you want extracted
- The AI figures out HOW to extract it
- New vendor format? The AI just reads it
- It's resilient — it handles variation naturally

This is why AI agents are a game-changer for **unstructured data**. Invoices, contracts, emails, support tickets — anything where the information is buried in natural language text.

> **As a PM:** Your job is designing the right tools and prompts, not writing parsing code. The AI handles the messy extraction. You design the structured output.

---

## Section 9: Experiment Corner — Break It and Learn!

Now it's your turn to tinker. Each experiment below changes ONE thing and lets you observe the effect. **Run each one and see what happens.**

### Experiment 1: Add a "category" field

What if we want the AI to automatically categorize each expense (office supplies, travel, food, software, etc.)? We just add a `category` field to the `add_expense` tool. The AI will figure out the categories on its own — we never tell it what the categories are!

In [ ]:
# Just run this cell! Press Shift+Enter
# Experiment 1: Adding a 'category' field — will the AI auto-categorize?

# Reset
expense_records_v2 = []

# New tool definitions with category added
TOOLS_V2 = [
    TOOLS[0],  # list_invoices (unchanged)
    TOOLS[1],  # read_invoice (unchanged)
    {
        "type": "function",
        "function": {
            "name": "add_expense",
            "description": "Record a single expense extracted from an invoice. Call this once per invoice after reading it.",
            "parameters": {
                "type": "object",
                "properties": {
                    "invoice_number": {"type": "string", "description": "The invoice number"},
                    "date": {"type": "string", "description": "The invoice date in YYYY-MM-DD format"},
                    "vendor": {"type": "string", "description": "The vendor/company name"},
                    "amount": {"type": "number", "description": "The total amount in rupees"},
                    "category": {"type": "string", "description": "The expense category (e.g. Office Supplies, Food & Dining, Travel, Software, Accommodation)"},
                },
                "required": ["invoice_number", "date", "vendor", "amount", "category"],
            },
        },
    },
    TOOLS[3],  # save_report (unchanged)
]


def add_expense_v2(invoice_number, date, vendor, amount, category):
    expense_records_v2.append({"invoice_number": invoice_number, "date": date, "vendor": vendor, "amount": amount, "category": category})
    return f"Recorded: Invoice #{invoice_number} | {vendor} | \u20b9{amount:,.0f} | Category: {category}"


def save_report_v2(filename):
    if not expense_records_v2:
        return "No expenses recorded yet."
    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["invoice_number", "date", "vendor", "amount", "category"])
        writer.writeheader()
        writer.writerows(expense_records_v2)
    return f"Saved {len(expense_records_v2)} expenses to {filename}"


def execute_tool_v2(name, args):
    if name == "list_invoices":
        return list_invoices()
    elif name == "read_invoice":
        return read_invoice(args["filename"])
    elif name == "add_expense":
        return add_expense_v2(args["invoice_number"], args["date"], args["vendor"], args["amount"], args["category"])
    elif name == "save_report":
        return save_report_v2(args["filename"])
    return f"Unknown tool: {name}"


# Run the agent with the new tools
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Please analyze all invoices and create an expense report CSV."},
]

print("Experiment 1: Agent with category field")
print("=" * 60)
print()

step = 0
while True:
    response = client.chat.completions.create(model=MODEL, max_tokens=MAX_TOKENS, tools=TOOLS_V2, messages=messages)
    message = response.choices[0].message
    messages.append(message)

    if message.tool_calls:
        for tc in message.tool_calls:
            step += 1
            name = tc.function.name
            args = json.loads(tc.function.arguments)
            result = execute_tool_v2(name, args)
            # Only show add_expense calls (the interesting part)
            if name == "add_expense":
                print(f"  Step {step}: {result}")
            messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
    else:
        break

print()
print("The AI auto-categorized every expense — we never told it what the categories should be!")

We added ONE field to the tool definition. The AI figured out the categories entirely on its own — "Office Supplies" for paper and ink, "Food & Dining" for the restaurant, "Travel" for the taxi. We didn't give it a list of categories. It just understood from context.

**PM takeaway:** Adding a field to a tool is like adding a column to a form. The AI fills it in intelligently.

---

### Experiment 2: Filter by amount

What if management only wants invoices above a certain amount? Let's change the system prompt to only process invoices over \u20b95,000.

In [ ]:
# Just run this cell! Press Shift+Enter
# Experiment 2: Only process invoices above Rs 5,000

expense_records_v3 = []


def add_expense_v3(invoice_number, date, vendor, amount):
    expense_records_v3.append({"invoice_number": invoice_number, "date": date, "vendor": vendor, "amount": amount})
    return f"Recorded: Invoice #{invoice_number} from {vendor} for \u20b9{amount:,.0f}"


def save_report_v3(filename):
    if not expense_records_v3:
        return "No expenses recorded."
    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["invoice_number", "date", "vendor", "amount"])
        writer.writeheader()
        writer.writerows(expense_records_v3)
    return f"Saved {len(expense_records_v3)} expenses to {filename}"


def execute_tool_v3(name, args):
    if name == "list_invoices":
        return list_invoices()
    elif name == "read_invoice":
        return read_invoice(args["filename"])
    elif name == "add_expense":
        return add_expense_v3(args["invoice_number"], args["date"], args["vendor"], args["amount"])
    elif name == "save_report":
        return save_report_v3(args["filename"])
    return f"Unknown tool: {name}"


# Changed system prompt — note the filter instruction
FILTERED_PROMPT = """You are an expense analyzer agent. Your job is to:

1. List all invoice files in the invoices/ directory
2. Read each invoice file one by one
3. Extract the key details: invoice number, date, vendor name, and total amount
4. ONLY record expenses where the total amount is ABOVE Rs 5,000. Skip smaller invoices.
5. After processing ALL invoices, save the report as "expense_report_filtered.csv"

Important: Extract the TOTAL amount (not subtotal). Use the vendor name from the "From:" line.
"""

messages = [
    {"role": "system", "content": FILTERED_PROMPT},
    {"role": "user", "content": "Please analyze all invoices and create a filtered expense report."},
]

print("Experiment 2: Only invoices above Rs 5,000")
print("=" * 60)
print()

step = 0
while True:
    response = client.chat.completions.create(model=MODEL, max_tokens=MAX_TOKENS, tools=TOOLS, messages=messages)
    message = response.choices[0].message
    messages.append(message)

    if message.tool_calls:
        for tc in message.tool_calls:
            step += 1
            name = tc.function.name
            args = json.loads(tc.function.arguments)
            result = execute_tool_v3(name, args)
            if name in ("add_expense", "save_report"):
                print(f"  Step {step}: {result}")
            elif name == "read_invoice":
                print(f"  Step {step}: Reading {args['filename']}...")
            messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
    else:
        break

print()
print(f"Result: {len(expense_records_v3)} invoices passed the Rs 5,000 filter (out of 5 total)")
print()
if message.content:
    print("Agent's note:", message.content)

We changed ONE line in the system prompt. The AI read every invoice, did the mental math, and **decided which ones to skip**. No `if amount > 5000` code needed. The AI applied the business rule from natural language instructions.

**PM takeaway:** Business rules live in the prompt, not the code. Changing requirements = changing a sentence.

---

### Experiment 3: Remove the save_report tool

What happens if the AI collects all the data but has no way to save it? Let's find out.

In [ ]:
# Just run this cell! Press Shift+Enter
# Experiment 3: What if we take away the save_report tool?

expense_records_v4 = []


def add_expense_v4(invoice_number, date, vendor, amount):
    expense_records_v4.append({"invoice_number": invoice_number, "date": date, "vendor": vendor, "amount": amount})
    return f"Recorded: Invoice #{invoice_number} from {vendor} for \u20b9{amount:,.0f}"


def execute_tool_v4(name, args):
    if name == "list_invoices":
        return list_invoices()
    elif name == "read_invoice":
        return read_invoice(args["filename"])
    elif name == "add_expense":
        return add_expense_v4(args["invoice_number"], args["date"], args["vendor"], args["amount"])
    return f"Unknown tool: {name}"


# Tools WITHOUT save_report
TOOLS_NO_SAVE = TOOLS[:3]  # Only list_invoices, read_invoice, add_expense

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Please analyze all invoices and create an expense report CSV."},
]

print("Experiment 3: Agent WITHOUT save_report tool")
print("=" * 60)
print()

step = 0
while True:
    response = client.chat.completions.create(model=MODEL, max_tokens=MAX_TOKENS, tools=TOOLS_NO_SAVE, messages=messages)
    message = response.choices[0].message
    messages.append(message)

    if message.tool_calls:
        for tc in message.tool_calls:
            step += 1
            name = tc.function.name
            args = json.loads(tc.function.arguments)
            result = execute_tool_v4(name, args)
            if name == "add_expense":
                print(f"  Step {step}: {result}")
            messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
    else:
        break

print()
print("=" * 60)
print("What did the agent say when it couldn't save?")
print("=" * 60)
if message.content:
    print(message.content)

Interesting! The AI processed all the invoices and recorded the expenses, but when it came time to save the CSV... it couldn't. It told us it was unable to save the file because it didn't have the tool.

**PM takeaway:** An agent is only as capable as its tools. Missing a tool = missing a capability. When designing an agent product, the tool inventory is the feature list.

---

## Section 10: PM Takeaways

You've just built and observed an AI agent that turns messy invoices into a clean expense report. Here's what to take away:

### Tool Design = Product Requirements
The fields you put in your tool definitions (what parameters, what types, what descriptions) directly control what the AI can do. Adding a "category" field made the AI auto-categorize expenses. Removing `save_report` made the AI unable to produce output. **Your tools are your feature spec.**

### System Prompt = Product Strategy
The system prompt tells the AI what to do and in what order. Changing one sentence ("only process invoices above Rs 5,000") changed the agent's entire behavior. **Your prompt is your product strategy document.**

### Agent Loop = Execution Engine
The loop is simple: ask the AI what to do, do it, repeat. The AI decides the order. It adapts to what it finds. It stops when it's done. **The loop is the engine; the tools and prompt are the steering wheel.**

### The Competitive Advantage
The difference between a great AI product and a failed POC often comes down to:
- **Well-designed tools** with clear descriptions and correct types
- **Specific system prompts** that leave no ambiguity
- **The right level of autonomy** — knowing when the AI should decide vs. when a human should

**Your competitive advantage as a PM:** knowing HOW agents work so you can design BETTER products. You don't need to write the code — but you need to understand the levers.